# P30-5-PNOF7

In [1]:
import yaml
import numpy as np

set_name = "P30-5"
nof = "PNOF7"
file = open('{}-{}.yaml'.format(set_name, nof), 'r')
data = yaml.safe_load(file)

In [2]:
def get_MADs(data):
    MADs = []
    for set in data:
        ADs = []
        for system in data[set]:
            ADs.append(data[set][system]["AD"])
        MADs.append(np.mean(ADs))
    return MADs

def get_Sets(data):
    Sets = []
    for set in data:
        Sets.append(set)
    return Sets

def get_Descriptions(subsets):
    f_des = open('Sets_description.yaml', 'r')
    d_des = yaml.safe_load(f_des)
    descriptions = []
    for set in subsets:
        descriptions.append(d_des[set])
    return descriptions

In [3]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [4]:
import math
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, TapTool
from bokeh.palettes import RdYlGn
from bokeh.transform import linear_cmap

subsets = get_Sets(data)
counts = get_MADs(data)
description = get_Descriptions(subsets)

source = ColumnDataSource(data=dict(subsets=subsets, counts=counts, description=description))

p = figure(x_range=subsets, width=1050, height=400, title="DietGMTKN55 {} {} Mean Absolute Deviation".format(set_name, nof))

p.xaxis.major_label_orientation = math.pi/2

p.vbar(x='subsets', top='counts', width=0.9, source=source,
       fill_color=linear_cmap('counts', palette=RdYlGn[11], low=min(counts), high=max(counts)), 
        line_color="white")

p.xaxis.axis_label = "Sets"
p.yaxis.axis_label = "MAD (kcal/mol)"
p.xgrid.grid_line_color = None

#p.add_tools(TapTool())
hover = HoverTool(tooltips=[
    ("Description", "@description"),
    ("MAD", "@counts"),
])
p.add_tools(hover)

show(p)

In [5]:
import ipywidgets as widgets
from IPython.display import display
import py3Dmol

def plot_molecules(data):

    subsets = get_Sets(data)
    nsets = len(subsets)
    dropdown_options = dict(zip(data.keys(), range(nsets)))
    descriptions = get_Descriptions(subsets)

    set_dropdown = widgets.Dropdown(
        options=dropdown_options,
        value=0,
        description='Set:'
    )

    set_box = widgets.VBox()

    set_stack = widgets.Stack()
    set_box.children = [set_dropdown, set_stack]

    widgets.jslink((set_dropdown, 'index'), (set_stack, 'selected_index'))

    boxes = []
    for h,set in enumerate(data):
        set_description = widgets.Text(
            value=descriptions[h],
            description='Description: ',
            style={'description_width': 'max-content'},
            disabled=False
        )

        system_box = widgets.VBox()

        system_label = widgets.Label(value="Systems (Reactions)")
        system_tabs = widgets.Tab()
        system_box.children = [set_description, system_label, system_tabs]

        system_tabs.children = [widgets.VBox() for system in data[set]]
        for i,system in enumerate(data[set]):
            system_tabs.set_title(i, str(system))

            mols_box = widgets.VBox()
            mols_label = widgets.Label(value="Molecules")
            mols_tabs = widgets.Tab()
            mols_box.children = [mols_label, mols_tabs]

            mols_tabs.children = [widgets.VBox() for specie in data[set][system]["Species"]]
            childrens = []
            for j,specie in enumerate(data[set][system]["Species"]):
                url = "url:https://raw.githubusercontent.com/stephengdale/poison-set/refs/heads/main/{}/{}.xyz".format(
                    set_name, specie
                )
                viewer = py3Dmol.view(url, width=280, height=300)
                viewer.setStyle({'stick': {'radius': .1}, 'sphere': {'scale': 0.25}})
                viewer.zoomTo()
                childrens.append(widgets.Output())
                with childrens[-1]:
                    viewer.show()
            mols_tabs.children = childrens
            for j,specie in enumerate(data[set][system]["Species"]):
                mols_tabs.set_title(j, str(specie))

            species_box = widgets.VBox([widgets.Label("Specie")] + [widgets.Label("{}".format(specie)) for specie in data[set][system]["Species"]])
            E_NOF_box = widgets.VBox([widgets.Label("E_NOF")] + [widgets.Label("{:>9.3f} Ha".format(data[set][system]["Species"][specie]["E_NOF"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            Stoichiometry_box = widgets.VBox([widgets.Label("Stoichiometry")] + [widgets.Label("{:^15}".format(data[set][system]["Species"][specie]["Count"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            # charge_box = widgets.VBox([widgets.Label("Charge")] + [widgets.Label("{}".format(data[set][system]["Species"][specie]["Charge"])) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            # mult_box = widgets.VBox([widgets.Label("Mult")] + [widgets.Label("{}".format(data[set][system]["Species"][specie]["Multiplicity"]+1)) for specie in data[set][system]["Species"]], layout=widgets.Layout(align_items='center'))
            text_specie_w = widgets.HBox([species_box, E_NOF_box, Stoichiometry_box])#, charge_box, mult_box])

            text_E_NOF_w = widgets.Text(
                value="{:.3f} kcal/mol".format(data[set][system]["dE_NOF"]*627.15),
                description='ΔE_NOF',
                disabled=False
            )

            text_E_ref_w = widgets.Text(
                value="{:.3f} kcal/mol".format(data[set][system]["dE_Ref"]),
                description='ΔE_Ref',
                disabled=False
            )

            # text_AD_w = widgets.Text(
            #    value="{:.3f} kcal/mol".format(data[set][system]["AD"]),
            #    description='Abs. Dev.',
            #    disabled=False
            # )

            # text_weight_w = widgets.Text(
            #    value="{:.3f}".format(data[set][system]["Weight"]),
            #    description='Weight',
            #    disabled=False
            # )

            # text_WTMAD2_w = widgets.Text(
            #    value="{:.3f}".format(data[set][system]["WTMAD2"]),
            #    description='WTMAD2',
            #    disabled=False
            # )

            system_tabs.children[i].children = [mols_box, text_specie_w,
                                             text_E_NOF_w, text_E_ref_w]#,
            # text_AD_w]#, text_weight_w,
            # text_WTMAD2_w]
        boxes.append(system_box)

    set_stack.children = boxes

    return set_box

In [6]:
nsets = len(list(data.items()))
step = nsets
for i in range(0, nsets, nsets):
    data_n = dict(list(data.items())[i:min(i+step,nsets)])
    tabs = plot_molecules(data_n)
    display(tabs)